In [13]:
import torch
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader, random_split, dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [14]:
Embed_dim = 300
word_num = 10_000
num_layers = 2
batch_size = 64

In [15]:
class Rnn(nn.Module):
    
    def __init__(self):
        super(Rnn, self).__init__()
        self.rnnL1 = nn.RNN(input_size=Embed_dim ,hidden_size=1024,
            nonlinearity= 'tanh' ,num_layers=num_layers, batch_first= False)
        self.linL2 = nn.Linear(1024,5)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        x=self.rnnL1(x)
        x=self.relu(self.linL2(x))
        x=nn.Softmax(x)
        
        return(x)
        

Dataset call section

In [16]:
dir_path = r"C:\Users\orian\OneDrive\שולחן העבודה\Coding Files 2\Datasets"
dataset = pd.read_csv(dir_path+r"\chat gpt reviews\ChatGPT_Reviews.csv")
emb_dict = pd.read_csv(dir_path+r"\embedding dictionary\dict(2).csv")

print(emb_dict.keys())

# Create a dataset dictionary of reviews and ratings
part_ds = [dataset['Review'], torch.tensor(dataset['Ratings'])]

temp_part_ds = []
# 
part_ds = list(zip(part_ds[0], part_ds[1]-1))

train_len = round(len(part_ds)*0.9)
train_ds_csv, test_ds_csv = random_split(
    dataset=part_ds,lengths=[train_len, len(part_ds)-train_len])
train_data = DataLoader(train_ds_csv, shuffle= True)

Index(['word'], dtype='object')


Embedding Section

In [20]:
word_dict = emb_dict['word'].tolist()
word_dict = set(word_dict)

# Separate the words in the reviews and make sure they don't repeat themselves
# (They are going to be added to the embeddings)
review_set = {word for sentence in dataset['Review'].astype(str)
               for word in sentence.split(' ')}

combined_words = word_dict.union(review_set)

# Dictionary to map words to indices
word_to_index = {key: val for val, key in enumerate(combined_words)}
embedding_layer = nn.Embedding(num_embeddings=len(word_to_index)
                               , embedding_dim=Embed_dim)

# Example of how to use the embedding layer that needs to be 
# deleted and moved to the feedforward section
input_words = ["suck","the","coconut","juice","out","of","the","coconut"]

# Made to check if the embedding layer works for 
# the training dataset here - it does. See if it gets distorted further down the line
tmp_random_input = list(train_data)[0][0]

# Strip the mentioned values out of each word in the embedded sentence and then turn them into a word array

tmp_random_input = str(tmp_random_input).strip("{}")
tmp_random_input = tmp_random_input.split(' ')
tmp_random_input = {word.strip('"(),''') for word in tmp_random_input}
print(tmp_random_input)

input_indices = torch.LongTensor([word_to_index[word] for word in input_words])  # Convert words to indices
embeddings = embedding_layer(input_indices)
print(embeddings.shape)


{'work', 'and', 'paid', 'a', 'but', 'busy', 'continually', 'very', 'it', "won't", 'here.', 'Tried', 'are', 'member', 'says', 'use', 'good', 'start', 'Not', 'the', 'to', 'servers'}
torch.Size([8, 300])


Single Epoch Training:

In [18]:
def train_one_epoch(rnn, train_ds, optimizer, dev, FILE_PATH, epoch_iter
                 # Should add stuff that's required
):
    iter = 0

    # Training loop part
    for sample in train_ds:
        rnn = rnn.to(dev) # Added due to errors (Should remain in the loop)
        print(sample[0])
        targetY = torch.nn.functional.one_hot(
            torch.tensor(sample[1]), num_classes=5)
        targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
        targetY = torch.squeeze(targetY).to(dev)
        
        # ---ForwardPropagation---
        
        embed_var = torch.LongTensor([word_to_index[word] for word in input_words])
        print(embed_var.shape)
        print(embed_var)
        pred = rnn.forward(embed_var).to(dev)
        loss_func = nn.CrossEntropyLoss()

        # ---BackPropagation---
        loss = loss_func(pred, targetY)
        loss.backward()
        optimizer.step()

        if iter%100==0:
            torch.save(rnn.cpu(), FILE_PATH)
            grad_tmp = rnn.linL2.weight.grad
            print(iter,"({})".format(epoch_iter+1), " | ",grad_tmp.sum()**2/len(grad_tmp))

        optimizer.zero_grad()
        iter+=1
    

Main Section

In [19]:
if __name__ == "__main__":
    if torch.cuda.is_available() : device = "cuda:0"
    else : device = "cpu"
    device = torch.device(device)
        
    rnn = Rnn().to(device)
    train_one_epoch(rnn, train_data, optimizer=optim.Adam(rnn.parameters(), lr=0.001), dev=device, FILE_PATH="model.pth", epoch_iter=0)
    

('best app you have to use it 😎😎',)
torch.Size([8])
tensor([227088,  75359, 366683, 462194,  37678, 472242,  75359, 366683])


C:\Users\orian\AppData\Local\Temp\ipykernel_13324\2179857343.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(sample[1]), num_classes=5)
C:\Users\orian\AppData\Local\Temp\ipykernel_13324\2179857343.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)


ValueError: RNN: Expected input to be 2D or 3D, got 1D tensor instead